```
Notebook Script for update grade control blast hole data
```

# Import necessary libraries

In [ ]:
import pandas as pd
from pathlib import Path
import csv
import shutil

## Setup variables

In [ ]:
date_file = '20251119'
file_name = "bh_462_midlusong_unblasted_225_20251119.csv"
dp_id = ["Mid_Lusong_225_Unblasted"]
bench = '225S'

In [ ]:
base = Path("C:/grade_control/01_grade_control")

# survey_holes = base / f"01_survey_holes/{date_file[:4]}/{date_file[:6]}/{date_file}"
# logbook = base / f"02_sampling/logbook/{date_file[:4]}/{date_file[:6]}"
# grade_samples = base / f"03_grade_samples/{date_file[:4]}/{date_file[:6]}/{date_file}"

In [ ]:
drill_plan = survey_holes / "drill_plan" / file_name
drill_plan

WindowsPath('D:/kworking/mr_directory/11_grade_control/01_survey_holes/2025/202511/20251111/drill_plan/bh_519_31_32b_lusong_225_t_rev1_20251111.csv')

In [ ]:
loaded_holes = survey_holes / "loaded_holes" / file_name
loaded_holes

WindowsPath('D:/kworking/mr_directory/11_grade_control/01_survey_holes/2025/202511/20251111/loaded_holes/bh_519_31_32b_lusong_225_t_rev1_20251111.csv')

# Initial validation of drill plan and loaded holes

## Drill Plan

In [ ]:
df_dp = pd.read_csv(drill_plan, skiprows=2, index_col=False, header=None)
df_dp.columns = ['string', 'y', 'x', 'z', 'd1', 'hole_id', 'd3', 'd4']
df_dp.columns = ['string', 'y', 'x', 'z', 'd1', 'hole_id', 'd3', 'd4']
df_dp = df_dp[df_dp["string"] != 0]
df_dp["hole_id"] = df_dp["hole_id"].astype(str).str.strip()
df_dp

## Loaded holes

In [ ]:
df_lh = pd.read_csv(loaded_holes, skiprows=2, index_col=False, header=None)
df_lh.columns = ["string", "y", "x", "z", "hole_id", "d2", "d3", "d4"]
df_lh = df_lh[df_lh["string"] != 0]
df_lh["hole_id"] = df_lh["hole_id"].astype(str).str.strip()
df_lh   

# Merge loaded holes with drill plan

In [ ]:
merged_df = pd.merge(df_lh, df_dp, on="hole_id", how="outer", suffixes=("_lh", "_dp"))
merged_df

In [ ]:
merged_df["d3"] = merged_df.apply(
    lambda row: row["d3_dp"] if row["d3_dp"] == row["d3_lh"] else row["d3_dp"], axis=1
)
merged_df["d4"] = merged_df.apply(
    lambda row: row["d4_dp"] if row["d4_dp"] == row["d4_lh"] else row["d4_dp"], axis=1
)

merged_df = merged_df.drop(
    columns=[
        "d3_dp",
        "d3_lh",
        "d4_dp",
        "d4_lh",
    ]
)
merged_df

## Offset Computation

Distance formula:

$d = \sqrt{(x_{2} - x_{1})^{2} + (y_{2} - y_{1})^{2}}$

Azimuth formula:

$\theta = \arctan2(\Delta x,\; \Delta y)$

Radians to Degrees conversion:

$\theta_{\text{deg}} = \theta \times \frac{180}{\pi}$

In [ ]:
# Delta x and Delta y
merged_df["dx"] = merged_df["x_lh"] - merged_df["x_dp"]
merged_df["dy"] = merged_df["y_lh"] - merged_df["y_dp"]

# Distance (Euclidean)
merged_df["distance"] = np.sqrt(merged_df["dx"] ** 2 + merged_df["dy"] ** 2)

# Azimuth (clockwise from north)
merged_df["azimuth"] = np.degrees(np.arctan2(merged_df["dx"], merged_df["dy"]))
merged_df["azimuth"] = merged_df["azimuth"] % 360

merged_df = merged_df.drop(columns=["dx", "dy"])           
merged_df

# Check distance of nearest holes where sampled holes were not surveyed 

In [ ]:
# checking = merged_df[merged_df['hole_id'].isin(["106", "105", "113", "119", "131", "135", "148", "159", "34", "36", "44", "45", "47", "50", "56", "59", "60", "61", "62", "63", "65", "65"])]
# checking

## Validation

### Duplicate Hole IDs

In [ ]:
dup_hole_ids = merged_df[merged_df.duplicated(subset="hole_id", keep=False)]
dup_hole_ids

## Locate nearest drill plan

In [ ]:
x_bad = dup_hole_ids.iloc[1]['x_lh']
y_bad = dup_hole_ids.iloc[1]["y_lh"]

print(x_bad, y_bad)

In [ ]:
tolerance = 10

merged_df["distance_to_bad"] = np.sqrt((merged_df["x_dp"] - x_bad) ** 2 + (merged_df["y_dp"] - y_bad) ** 2)
possible_matches = merged_df[merged_df["distance_to_bad"] <= tolerance]
nearest_hole = possible_matches.loc[possible_matches["distance_to_bad"].idxmin()]
print("Nearest hole:", nearest_hole["hole_id"])

## Missing rows

In [ ]:
# Checking rows with NaN values
nan_rows = merged_df[merged_df.isna().any(axis=1)]
nan_rows

### Loaded Holes ✅ | Drill Plan ❌

Holes below are additional holes

In [ ]:
# Missing holes in drill plan but loaded
missing_in_dp = merged_df[merged_df["x_dp"].isna()]
missing_in_dp

### Loaded Holes ❌ | Drill Plan ✅

Holes below are not drilled

In [ ]:
# Holes in loaded but not in drill plan
missing_in_lh = merged_df[merged_df["x_lh"].isna()]
missing_in_lh

# Cleaned merged data

Make sure the survey data is cleaned before running script below

# Import Logbook.csv

In [ ]:
df_lb = pd.read_csv(f"{logbook}/logbook.csv")
df_lb = df_lb[df_lb["Drill Plan"] == dp_id]

# Validation: if Sample ID contains 'CR' or 'BLANK' (case-insensitive) set 'Drill Plan' to blank
if 'Sample ID' in df_lb.columns and 'Drill Plan' in df_lb.columns:
    mask = df_lb['Sample ID'].astype(str).str.contains(r'\b(?:CR|DUP|BLANK)\b', case=False, na=False)
    df_lb.loc[mask, 'Drill Plan'] = ''

df_lb

## Final Cu

In [ ]:
df_lb['Cu_final'] = df_lb['Cu_Reassay'].fillna(df_lb['Cu_Orig'])
df_lb = df_lb.drop(columns=['Cu_Orig', 'Cu_Reassay'])

df_lb

# Final dataframe | Merged survey and assay data

In [ ]:
df_final = pd.merge(merged_df, df_lb, left_on='hole_id', right_on='Sample ID', how='outer')
df_final = df_final[df_final['Unique ID'].notna()]
df_final['d4'] = date_file
df_final['Cu_final'] = pd.to_numeric(df_final['Cu_final'])
df_final

# Validation

## Check samples with no loaded holes

In [ ]:
cols = ['y_lh', 'x_lh', 'z_lh']
missing_hole_id = df_final[df_final[cols].isna().any(axis=1)]
missing_hole_id

# Convert to grade samples

In [ ]:
# Filter columns 
df_final['BHID'] = df_final['d4'] + "_" + bench + "_" + df_final['hole_id']

filtered_cols = ['string_lh', 'y_lh', 'x_lh', 'z_lh', 'Cu_final', 'hole_id', 'd3', 'd4', 'BHID',  'Drill Plan', 'z_dp', 'Unique ID', 'Sample ID','CuO_Soluble', 'Au', 'Ag']
df_grade_samples = df_final[filtered_cols].copy()

bins = [0, 0.119, 0.179, 0.249, 0.299, 0.399, 0.599, 999]
labels = [1, 2, 3, 4, 5, 6, 7]

# Create the 'string_lh' column based on 'Cu_final'
df_grade_samples['string_lh'] = pd.cut(
    df_grade_samples['Cu_final'],
    bins=bins,
    labels=labels,
    include_lowest=True
)

gs_file = grade_samples / file_name

df_grade_samples.to_csv(gs_file, index=False)
df_grade_samples

# Convert back to string files

In [ ]:
# Read original CSV
with open(gs_file, newline='') as f:
    reader = list(csv.reader(f))

# Remove the first row
reader = reader[1:]

# Add rows at top and bottom
new_rows = [
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0]
] + reader + [
    [0, 0, 0, 0],
    [0, 0, 0, 0, "END"]
]

# Write back to CSV
with open(gs_file, "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerows(new_rows)


shutil.copy(gs_file, f"{gs_file.parent}/{gs_file.stem}.str")